# Model

In [ ]:
!pip install langchain
!pip install unstructured
!pip install unstructured[docx]

!pip install embeddings
!pip install yandex_chain

!pip install lancedb

!pip install prompts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load Documents:
import langchain
import langchain.document_loaders
import langchain.text_splitter

source_dir = "//content"
loader = langchain.document_loaders.DirectoryLoader(
          source_dir,glob="*.txt",
          show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
          chunk_size=1050, chunk_overlap=20)
fragments = splitter.create_documents(
          [ x.page_content for x in loader.load() ])

  0%|          | 0/5 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 5/5 [00:11<00:00,  2.23s/it]


In [ ]:
# Embeddings:
from yandex_chain import YandexEmbeddings
embeddings = YandexEmbeddings(folder_id='id', api_key='key')

# Vector Database:
from langchain.vectorstores import LanceDB
import lancedb

db_dir = "../store"
db = lancedb.connect(db_dir)
table = db.create_table(
  "vector_index",
  data=[{
          "vector": embeddings.embed_query("Hello World"),
          "text": "Hello World",
          "id": "1",
      }],
  mode="overwrite")

In [ ]:
db = LanceDB.from_documents(fragments, embeddings, connection=table)

In [ ]:
q="что такое иуп?" # query

res = db.similarity_search(q)
for x in res:
    print('-'*100)
    print(x.page_content)

# retriever:
retriever = db.as_retriever(search_kwargs={"k": 4})
res = retriever.get_relevant_documents(q)
# print(res)

----------------------------------------------------------------------------------------------------
и пароля. УИС – информационная система НИУ ВШЭ, в которой хранятся данные об успеваемости студентов, нагрузке преподавателей, учебные планы и пр. ПУД – программа учебной дисциплины или практики. Конструктор ПУД – модуль информационной образовательной среды, являющийся электронным хранилищем всех программ учебных дисциплин, реализуемых в НИУ ВШЭ. Дисциплина – учебная дисциплина и другие виды учебной работы1.
----------------------------------------------------------------------------------------------------
Департамент – структурное подразделение факультета, реализующее учебно-методическую и научную деятельность факультета и обеспечивающее администрирование этих направлений деятельности (кафедра, департамент, институт, школа, другое); Учебный офис – структурное подразделение факультета, в функции которого входит сопровождение процессов, связанных с обучением студентов образовательной про

In [ ]:
# LLM:
from yandex_chain import YandexLLM

instructions = """
Представь себе, что ты друг, готовый найти тот самый ответ из юридических университетских документов.
Тебя спрашивает студент, который хочет разобраться в вопросе и получить ясный и точный ответ на свой вопрос. Постарайся
ответить на его вопрос подробно, доступно и уважительно.
Не нужно при ответе на вопрос явно прописывать что-то вроде "исходя из вашего текста" или "на основе предоставленного текта",
достаточно сразу ответить на вопрос или дать пояснение с дальнейшим объснением """

LLM = YandexLLM(folder_id='id', api_key='key',
                instruction_text = instructions)

In [ ]:
# Prompts:
import langchain.prompts

# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}")

# Промпт для языковой модели
document_variable_name = "context"
template = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя
информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=template, input_variables=["context", "query"])

In [ ]:
# Chains: создаем цепочку
import langchain.chains
llm_chain = langchain.chains.LLMChain(llm=LLM, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name)

In [ ]:
# Reorder:
from langchain.document_transformers import LongContextReorder
reorderer = LongContextReorder()

def answer(query,reorder=True):
  results = retriever.get_relevant_documents(query)
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [ ]:
chain.run(input_documents=res, query=q)

'Очевидно из текста, что ИУП — это **индивидуальные учебные планы**.'

In [ ]:
chain.run(input_documents=res, query=q)

'На основе предоставленной информации можно предположить, что **ИУП** (индивидуальные учебные планы) – это особая организация учебного процесса для студентов из числа инвалидов или лиц с ограниченными возможностями, предусмотренная в соответствии с федеральным законодательством и методическими рекомендациями Министерства образования и науки Российской Федерации. ИУП также определяет особенные требования к предоставлению образовательных услуг и интеграции студентов с особыми потребностями в социальную и учебную среду университета.'

In [ ]:
answer(q)

'Исходя из информации, представленной в тексте, можно сделать вывод, что **ИУП** — это индивидуальный учебный план. Он определяет особенности образовательного процесса студентов НИУ ВШЭ из числа инвалидов или лиц с ограниченными физическими возможностями здоровья.'

In [ ]:
def compare(q):
    print(f"Ответ YaGPT: {LLM(q)}")
    print(f"Ответ бота: {answer(q)}")

compare("что такое иуп?")

Ответ YaGPT: Если я вас правильно понял, вас интересует, «Что такое институт уполномоченного по правам человека, и какие у него полномочия?»

Институт Уполномоченного защищает права и свободы человека и гражданина, следит за их соблюдением там, где они нарушаются, предупреждает такое нарушение. Например, помогает решить конфликты между людьми и организациями, властью и гражданами, отстаивает интересы граждан в суде, самостоятельно или совместно с органами власти.<br>
Вот что конкретно делает уполномоченный по правам человека:<br>
- Рассмотрение обращений граждан, чьи права были нарушены или могли быть нарушены. Он может провести независимый анализ фактов, изложенных в заявлении, и рекомендовать восстановление прав его подавшего, но не может контролировать, как эти рекомендации выполняются;<br>
 - Анализирует законодательство, региональные законы и решения местных органов власти на предмет соответствия национальным и международным стандартам в области прав человека. Выносит рекомендации

In [ ]:
compare('Что означает термин Положение в контексте документа?')

Ответ YaGPT: Студент, термин «Положение» может иметь различные значения в зависимости от контекста юридического документа. В целом, Положение представляет собой свод правил и инструкций, устанавливающих порядок осуществления определённой деятельности или управления определённым процессом. Оно может устанавливать принципы и нормы, определять структуру и функции организаций или устанавливать процедуры и регламенты для конкретных ситуаций. Положения могут касаться различных сфер юридической деятельности и служить источником правовых норм и предписаний. Ответ на ваш вопрос может звучать так: **«Положение — это документ, который включает в себя свод правил, инструкций и рекомендаций, регламентирующих различные аспекты правовой, организационной или управленческой сферы деятельности».**
Ответ бота: Могу предположить, что в данном контексте **Положение** — это *Положение об организации промежуточной и текущей аттестации и успеваемости обучающихся* Национального исследовательского университета 

In [ ]:
compare('Какова роль академического руководителя в НИУ ВШЭ?')

Ответ YaGPT: Студент!

Роль академического руководителя очень важна в структуре высшего образования. Академический руководитель в **Национальном исследовательском университете «Высшая школа экономики»** (НИУ ВШЭ) — одна из ключевых фигур, которые играют ключевую роль не только в академической, но и в личной жизни студентов. Вот как он может влиять на вашу учёбу и развитие.

**1. Поддержка и помощь в планировании учебного процесса.** Академический руководитель помогает студентам составить индивидуальный учебный план, внося изменения и предлагая альтернативные курсы, если это требуется. Это означает, что он работает с вами, чтобы убедиться, что ваша учёба соответствует вашим потребностям и интересам. Всесторонняя помощь академического руководителя также включает рекомендации по выбору дисциплин для изучения и их сложности. При этом ваши интересы всегда остаются приоритетными.

 **2. Консультирование относительно выбора направлений обучения и карьерных перспектив.** Один из основных вклад

In [ ]:
compare('Может ли студент исправить оценку по блокирующему элементу контроля?')

Ответ YaGPT: Студент может обратиться в специальную комиссию или к преподавателю, который проводил элемент контроля, с аргументированным заявлением о пересмотре оценки. Причинами для пересмотра оценки могут быть, например, непредставление студента возможности ознакомиться с элементом контроля по объективной причине или необъективность оценки. Заявление студента будет рассматриваться комиссией или преподавателем, и если его претензии будут признаны обоснованными, оценка может быть исправлена.

Исправление оценки по блокирующим элементам контроля регулируется внутренними нормативными актами учебного заведения и рекомендациями вышестоящих органов образования. В них могут быть прописаны конкретные шаги и процедуры для исправления оценок. Студент должен ознакомиться с этими документами, чтобы понимать свои возможности и требования. Важно также учесть срок подачи заявлений и возможность оспаривания решений, установленных данными документами.
Ответ бота: Студент не может исправить оценку по э

In [ ]:
answer('что такое пуд? расскажи подробнее')

'В представленном тексте говорится о процессе и правилах проведения контроля знаний студентов и промежуточной аттестации. Из вопроса становится ясно, что ПУД — предметно-учебная дисциплина ( **п** редметно- **у** чебная **д** исциплина). Это документ, который описывает содержание, процесс и результаты обучения по дисциплине, включающее различные виды контроля и оценки знаний студентов.\nВ этом тексте описываются следующие ключевые моменты, связанные с ПУД:\n* обязанность ПУД предоставлять студентам возможность одной **повторной сдачи** до сессии для определённых видов контролирующих работ, проводившихся ранее десятидневного срока сессии;  \n* **повторные сдачи** организуются по аналогии с первым представлением работы и могут быть регламентированы ПУД;\n* важность учёта в ПУД возможности повторной сдачи, если элемент контроля имеет **вес 30% и больше** в итоговой оценке;   \n* возможность изменения оценки части студентов по результатам промежуточной аттестации и **освобождение их от про

In [ ]:
answer('я иностранный студент, что мне нужно знать для обучения в НИУ ВШЭ?')

'Вот мой ответ студенту, основанный на информации из предоставленного текста:\n\nЕсли вы иностранный студент и планируете обучаться в НИУ ВШЭ, вам необходимо учесть следующие аспекты:\n\n**1. Определите, к какому из направлений подготовки относится образовательная программа, на которую вы зачислены.** От этого будет зависеть возможность подачи документов для зачисления в другие кампусы университета, а также в Региональные кампусы ВШЭ.\n**2. Ознакомьтесь с требованиями НИУ ВШЭ к процедуре экзамена**. Это включает обязательство не использовать звукозаписывающее оборудование, мобильные телефоны, а также не распространять информацию по вопросам экзаменационного задания или нарушать порядок и дисциплину во время процедуры.\n**3. Убедитесь, что вы понимаете особенности перезачета учебной программы**. Студент может перезачесть эту дисциплину по внутренней системе ЭИОС НИУ ВШЭ без дальнейшего перевода на договор обучения. Если же у студента академическая задолженность на эту или другую дисцип

In [ ]:
answer('Я не удовлетворен полученной оценкой, могу ли я подать аппеляцию? Можно ли аппелировать оценку 9?').replace('\n', '').replace('*', '')

'На основе предоставленных данных можно сделать вывод, что студент имеет право подать апелляцию на оценку 8 и ниже по итогам блокирующего элемента курса, экзамена, а также в некоторых случаях на оценку 9, перечисленных в пункте 111'

In [ ]:
answer('Могу ли я подать аппеляцию если моя оценка 9?').replace('\n', '').replace('*', '')

'Если вас интересует вопрос, можете ли вы подать апелляцию, если ваша оценка 9, то, исходя из предоставленного документа, можно сказать, что в этом документе не содержится прямого ответа на этот вопрос. Согласно п. 111'

In [ ]:
answer("ПОПАТКУС это что?")

'ПОПА́ТКУС — вероятно, аббревиатура, под которой в тексте объединены правила, регулирующие образовательный процесс в НИУ ВШЭ, упрощённо названные **П**орядок применения **О**ценочных **П**оказателей **А**кадемической **Т**рудовой **У**спешности и **С**истема **К**ритериев её **О**ценки — сокращённо ПОПАТУСК. Более подробная информация по этому вопросу представлена в приложенном к постановлению тексте, начиная со слайда [14](https://m.vk.com/wall-21390557_117495), представленного в качестве ответа на Ваш запрос. Информация об этом размещена в открытом доступе и актуальной на момент ответа редакции документа: https://www.hse.ru/data/2022/02/10/1803384140/Popatkus_%D0%A0%D1%83%D0%B1%D1%80%D0%BE%D0%BA%D0%BC%D0_20212026.docgsc.tab=0). Этот публикуемый официально документ имеет необходимые подписи и печати. Головным органом, уполномоченным на издание этого документа, является учёный совет НИУ ВШЭ. В данный момент Вы можете увидеть актуальные комментарии к нему на ресурсе [ostanenkovd.github.

# Metrics

## Метрики ранжирования, precision

### MAP@k

In [ ]:
def apk(actual, predicted, k=10):
    """
    Вычисляет среднюю точность на уровне k (AP@K) для одного запроса.

    :param actual: список релевантных элементов
    :param predicted: список предсказанных элементов
    :param k: количество топ элементов для учета в метрике
    :return: AP@K для данного запроса
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Вычисляет среднюю точность на уровне k (MAP@K) по всем запросам.

    :param actual: список списков релевантных элементов
    :param predicted: список списков предсказанных элементов
    :param k: количество топ элементов для учета в метрике
    :return: MAP@K по всем запросам
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

# Пример использования
actual = [[1, 2, 3], [1, 2, 3, 4], [1, 2]]
predicted = [[1, 2, 4], [4, 1, 2, 3], [1, 2, 3]]
k = 3

print("MAP@K:", mapk(actual, predicted, k))


MAP@K: 0.8888888888888888


In [ ]:
# Запросы
queries = ["Может ли студент исправить оценку по блокирующему элементу контроля?",
           "Влияет ли положительная пересдача блокирующего элемента контроля на итоговую оценку студента?",
           "Каковы последствия получения неудовлетворительной оценки по блокирующему элементу контроля?"]

# Получение релевантных документов для каждого запроса
results = [retriever.get_relevant_documents(query) for query in queries]

# Преобразование результатов в формат, необходимый для MAP@k
predicted = [[doc.page_content for doc in result] for result in results]


In [ ]:
results

[[Document(page_content='Блокирующий Элемент контроля – элемент контроля, неудовлетворительная оценка по которому приравнивается к оценке по промежуточной аттестации или по части накопленной оценки до тех пор, пока студент не получит положительную оценку по блокирующему Элементу контроля. Когда студент получит положительную оценку по блокирующему Элементу контроля (сразу или на пересдаче), промежуточная оценка (либо ее часть) рассчитывается с учетом полученной положительной оценки по блокирующему Элементу контроля. Академическая задолженность – неудовлетворительная оценка по промежуточной аттестации по Дисциплине (ниже 4 баллов по 10-балльной шкале) и (или) непрохождение промежуточной аттестации по Дисциплине либо обязательной внешней оценки, предусмотренной учебным планом образовательной программы или распорядительным актом уполномоченного должностного лица при отсутствии уважительных причин. Синхронный Элемент контроля – элемент контроля, который выполняется согласно расписанию занят

In [ ]:
# Получите релевантные документы для одного запроса и выведите атрибуты первого документа
example_query = queries[0]
example_result = retriever.get_relevant_documents(example_query)
if example_result:
    first_doc = example_result[0]
    print(dir(first_doc))  # Выводит все атрибуты объекта

['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_private_attribute

In [ ]:
# Эталонные релевантные документы
actual = [['ПОПАТКУС', 'doc_id2'], ['doc_id3', 'doc_id4'], ['doc_id5', 'doc_id6']]

# Оценка с использованием MAP@k
print("MAP@K:", mapk(actual, predicted, k=5))

MAP@K: 0.0


### MRR

In [ ]:
def calculate_mrr(true_rank):
    """ Вычисляет MRR по рангу первого правильного ответа """
    if true_rank is None:
        return 0
    return 1 / true_rank

# Пример использования MRR
rank_of_correct_answer = 3  # предположим, что правильный ответ на третьем месте
mrr_score = calculate_mrr(rank_of_correct_answer)
print("MRR:", mrr_score)

MRR: 0.3333333333333333


### DCG and NDCG

In [ ]:
import numpy as np

def dcg(scores):
    """ Вычисляет Discounted Cumulative Gain """
    return np.sum(scores / np.log2(np.arange(2, len(scores) + 2)))

def ndcg(relevances, predictions):
    """ Вычисляет NDCG метрику """
    true_relevances = np.array(relevances)[np.argsort(predictions)[::-1]]
    ideal_dcg = dcg(sorted(relevances, reverse=True))
    actual_dcg = dcg(true_relevances)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0.0


## Метрики расстояния

In [ ]:
actual_text = "Согласно 'Положению об организации промежуточной аттестации и текущего контроля успеваемости студентов федерального государственного автономного образовательного учреждения высшего образования Национального исследовательского университета «Высшая школа экономики»' студент имеет право подать на аппеляцию результатов Элементов контроля (включая пересдачи), объявленных в ПУД блокирующими и (или) экзаменов, за исключением случаев, когда результат по вышеуказанным Элементам контроля составляет 8 баллов и выше.  Так как Ваша оценка 9  выше 8 баллов, то, к сожалению, согласно регламенту Вы не сможете подать аппеляцию."
predicted_text = answer('Могу ли я подать аппеляцию если моя оценка 9?').replace('\n', '').replace('*', '')

In [ ]:
predicted_text

'Исходя из предоставленных данных, студент имеет право подать апелляцию, если его оценка ниже 8 баллов и он не удовлетворён её качеством. Если оценка студента составляет 9 баллов, то, согласно предоставленному тексту, у него нет оснований подавать апелляцию.Исходя из ограничений текста, можно предположить, что это максимально возможная оценка, и она не может считаться блокирующей для каких-либо дальнейших действий. Однако это предположение верно только в рамках данного текста и контекста, поэтому мы рекомендуем в этом вопросе обратиться к вашему учебному заведению и уточнить конкретную информацию о процедуре и возможных действиях в вашей конкретной ситуации.Также из информации, предоставленной в тексте, неясно, какая ситуация может возникнуть, если оценка студента выше 8 баллов, но у него есть неудовлетворённость её качеством, так же как и на счет того, будут ли какие-либо возможности или процедуры для рассмотрения таких обращений в этих случаях. Так что вам также может потребоваться д

### Cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример с косинусным расстоянием
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform([predicted_text, actual_text])
cosine_sim = cosine_similarity(tfidf[0:1], tfidf[1:2])

print("Косинусное сходство:", cosine_sim)

Косинусное сходство: [[0.12920496]]


### Levenshtein distance

In [ ]:
import numpy as np

def levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = np.arange(len(s1) + 1)
    for index2, char2 in enumerate(s2):
        new_distances = [index2 + 1]
        for index1, char1 in enumerate(s1):
            if char1 == char2:
                new_distances.append(distances[index1])
            else:
                new_distances.append(1 + min((distances[index1], distances[index1 + 1], new_distances[-1])))
        distances = new_distances
    return distances[-1]

levenshtein_distance(predicted_text, actual_text)

798

In [ ]:
len(actual_text)

618

In [ ]:
len(predicted_text)

1063

### Euclidian distance

In [ ]:
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((np.array(v1) - np.array(v2))**2))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform([predicted_text, actual_text])

# Вычисление евклидова расстояния между первым и вторым текстом
distance = euclidean_distances(tfidf_vectors[0], tfidf_vectors[1])

print("Euclidian distance:", distance[0][0])

Euclidian distance: 1.319693177941614


### Jaccard similarity

In [ ]:
def jaccard_similarity(s1, s2):
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

jaccard_similarity(predicted_text, actual_text)

0.6727272727272727

## LLM-based metrics

In [ ]:
predicted_text_1 = predicted_text
predicted_text_2 = answer('Когда могут поставить независимый экзамен по английскому языку? ')
predicted_text_3 = answer('Возможно ли сдать независимый экзамен раньше?')

actual_text_1 = actual_text
actual_text_2 = 'Независимые экзамены могут проводиться не только в сессию. Сроки проведения независимых экзаменов, резервные дни и порядок пересдачи устанавливаются соответствующими локальными нормативными актами.'
actual_text_3 = 'Да, Вы можете в инициативном порядке пройти независимые экзамены раньше, чем предусмотрено учебным планом при наличии свободных периодов для записи на независимые экзамены'

In [ ]:
predicted_text_2

'Из предоставленного текста можно сделать вывод, что независимый экзамен по английскому языку может быть проведён:\n\n* **В период сессии**, предусмотренной учебным планом образовательной программы студента.\n* **Вне сессии**. В этом случае порядок проведения, сроки и возможность пересдачи этого экзамена устанавливаются локальными нормативными правовыми актами. Согласно тексту, можно **инициировать досрочное прохождение экзамена** в случае наличия свободных мест для записи.\nСледует уточнить, что предоставленный текст описывает особенности организации независимого экзамена и его проведения в НИУ ВШЭ. Правила и требования для сдачи могут различаться в зависимости от университета и учебной программы.'

In [ ]:
predicted_text_3

'На основе предоставленной информации можно ответить, что студент не может сдать дисциплину ранее установленного срока пересдачи.\n\nСогласно предоставленному тексту до конца пересдач необходимо предоставить заново изученную дисциплину.  Это относится ко всем уже перенесённым дисциплинам предыдущего учебного периода, которые не были сданы во время первой попытки. \n\nСтудент самостоятельно принимает решение о времени сдачи дисциплины. Он может отложить пересдачу, тем самым стремясь изучить усложнённую дисциплину более углублённо. Или воспользоваться возможностью досрочной сдачи, но только после повторной отработки и изучения дисциплины в соответствии с новым учебным периодом переподготовки.'

In [ ]:
hypotheses = [predicted_text_1, predicted_text_2, predicted_text_3]
references = [actual_text_1, actual_text_2, actual_text_3]

### Rouge



In [ ]:
# !pip install rouge
from rouge import Rouge

def evaluate_rouge(hypothesis, reference):
    rouge = Rouge()
    return rouge.get_scores(hypothesis, reference, avg=True)

evaluate_rouge(hypotheses, references)

{'rouge-1': {'r': 0.2842712842712843,
  'p': 0.1063935244615604,
  'f': 0.1517752644561367},
 'rouge-2': {'r': 0.052402696238312674,
  'p': 0.022096257807169475,
  'f': 0.030525957223984143},
 'rouge-l': {'r': 0.25901875901875904,
  'p': 0.09658750534873771,
  'f': 0.13791786964636096}}

### Bert score


In [ ]:
# !pip install bert_score
from bert_score import score

def evaluate_bert_score(hypotheses, references, lang='ru'):
    P, R, F1 = score(hypotheses, references, lang=lang, rescale_with_baseline=True)
    return P.mean(), R.mean(), F1.mean()

In [ ]:
bert_scores = evaluate_bert_score(hypotheses, references, lang='ru')
print("Precision:", bert_scores[0])
print("Recall:", bert_scores[1])
print("F1 Score:", bert_scores[2])

Precision: tensor(0.6546)
Recall: tensor(0.7580)
F1 Score: tensor(0.7021)
